In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel(r"C:\Users\ljx\Desktop\121.xlsx",index_col="date")

In [3]:
nan_count = df.isna().sum().sum()

# 打印NaN的数量
print(f"缺失值（NaN）的数量: {nan_count}")

缺失值（NaN）的数量: 337


In [4]:
df_filled_ffill = df.fillna(method='ffill')
nan_count = df_filled_ffill.isna().sum().sum()

# 打印NaN的数量
print(f"缺失值（NaN）的数量: {nan_count}")

缺失值（NaN）的数量: 8


In [5]:
df_filled_bfill = df_filled_ffill.fillna(method='bfill')
nan_count = df_filled_bfill.isna().sum().sum()

# 打印NaN的数量
print(f"缺失值（NaN）的数量: {nan_count}")

缺失值（NaN）的数量: 0


In [6]:
df=df_filled_bfill
nan_count = df.isna().sum().sum()

# 打印NaN的数量
print(f"缺失值（NaN）的数量: {nan_count}")

缺失值（NaN）的数量: 0


In [7]:
from scipy.optimize import minimize
# 计算每日收益率
daily_returns = df.pct_change().dropna()

def objective_function(weights, returns, risk_free_rate):
    # 计算投资组合的夏普比率，最大化夏普比率等于最小化相反的夏普比率
    portfolio_return = np.sum(returns.mean() * weights) * 252  # 年度平均，假设252个交易日
    excess_return = portfolio_return - risk_free_rate
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe_ratio = excess_return / portfolio_volatility
    return -sharpe_ratio  # 最小化相反的夏普比率

# 初始权重猜测值
initial_weights = np.ones(len(df.columns)) / len(df.columns)

# 定义权重的约束（权重总和为0.95）
target_weight_sum = 0.95
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - target_weight_sum})

# 定义权重的上下限（以百分比表示，保留两位小数）
lower_bound = 0.0
upper_bound = 0.05
bounds = [(lower_bound, upper_bound) for _ in initial_weights]

# 假设无风险利率为3.91%
risk_free_rate = 0.0391

# 使用 minimize 函数找到最小值（最大化夏普比率）
result_with_risk_free = minimize(objective_function, initial_weights, args=(daily_returns, risk_free_rate),
                                  method='SLSQP', bounds=bounds, constraints=constraints)

# 输出优化结果
optimal_weights = result_with_risk_free.x
max_sharpe_ratio = -result_with_risk_free.fun

# 计算最优权重对应的投资组合的预期年收益和年波动率
optimal_portfolio_return = np.sum(daily_returns.mean() * optimal_weights) * 252
optimal_portfolio_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(daily_returns.cov() * 252, optimal_weights)))

# 输出结果
print("Optimal Weights (in percentage):", [f"{w*100:.2f}%" for w in optimal_weights])
print("Max Sharpe Ratio:", max_sharpe_ratio)
print("Expected Annual Return of Optimal Portfolio:", optimal_portfolio_return)
print("Annual Volatility of Optimal Portfolio:", optimal_portfolio_volatility)

# 统计约等于两位小数之后仍显示为非零数的个数
non_zero_weights_count = np.sum(np.abs(optimal_weights) >= 1e-4)  # 1e-4 是一个小的阈值

# 输出结果
print("Number of Non-Zero Weights (approximately two decimal places):", non_zero_weights_count)

Optimal Weights (in percentage): ['0.00%', '0.00%', '0.81%', '4.37%', '0.00%', '0.00%', '2.46%', '0.21%', '5.00%', '0.00%', '0.00%', '0.00%', '0.00%', '4.14%', '5.00%', '0.00%', '1.84%', '0.00%', '0.00%', '0.00%', '5.00%', '5.00%', '0.00%', '0.00%', '0.00%', '2.56%', '0.00%', '0.00%', '0.00%', '0.00%', '5.00%', '3.39%', '0.00%', '0.00%', '0.00%', '3.86%', '0.00%', '1.57%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.08%', '0.00%', '1.49%', '0.00%', '0.00%', '4.27%', '5.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '5.00%', '0.00%', '0.00%', '0.00%', '0.00%', '5.00%', '5.00%', '0.00%', '0.00%', '0.00%', '5.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.00%', '0.

In [9]:
# 将权重添加到 DataFrame 中
df_new = pd.DataFrame({'name': df.columns.values, 'weights': optimal_weights})
df_new
df_new.to_excel(r"C:\Users\ljx\Desktop\28.xlsx", index=False)